# PerStruc-Denoiser

In [ ]:
# Jupyter-notebook magic

# For the matplotlib 
%matplotlib inline
# For reload functions explicitly
%load_ext autoreload
%autoreload 2

In [ ]:
# Imports

## Add the modules to the system path
import os
import sys
sys.path.append(os.path.join(".."))

## Libs
import glob
import tifffile
import matplotlib.pyplot as plt
import numpy as np

## Own modules
from perstruc_denoiser import perstruc_denoiser
import utils

## Folder Structures

In [ ]:
# Enter the store path for the results and denoised file (with periodic structured noise) here #
path_results = os.path.join("..", "results_PerStruc-Denoiser")
path_dataset = os.path.join("..", "data", "PerStruc-Denoiser")
#********************************************************#

In [ ]:
# Create a folder for the PerStruc-Denoiser results folder

# Make a folder to store the PerStruc-Denoiser
os.makedirs(path_results, exist_ok=True)

## Load data

In [ ]:
## Load image stack 
filenames_raw = glob.glob(os.path.join(path_dataset, "*-raw.tif"))
filenames_denoised = glob.glob(os.path.join(path_dataset, "*-denoised.tif"))
print("Following denoised file will be PerStruc-denoised:  ", filenames_denoised[0])

file_raw = tifffile.imread(filenames_raw[0])
file_denoised = tifffile.imread(filenames_denoised[0])

## Check the periodic structured noise

In [ ]:
# Example image
file_example_denoised = file_denoised[file_denoised.shape[0]//2]

# Autocorrelation
example_autocorrelation = utils.autocorrelation(file_example_denoised)

# Line power spectrum
freqs, example_line_spectrum = utils.line_power_spectrum(file_example_denoised)

In [ ]:
plt.figure(figsize=(30, 10))

plt.subplot(311)
plt.title("Example image (3PM-N2V)")
plt.imshow(file_example_denoised, cmap="Greens")


plt.subplot(312)
plt.title("Autocorrelation (3PM-N2V) of center patch")
plt.imshow(example_autocorrelation, cmap="magma")

plt.subplot(313)
plt.title("Averaged line power spectrum (3PM-N2V)")
plt.plot(freqs, example_line_spectrum, linewidth = 5, label="3PM-Noise2Void")
plt.ylabel("Normlized spectral power")
plt.xlabel(r"Spatial frequency [$Pixel^{-1}$]")
plt.legend()

plt.tight_layout
plt.show()

## Detect the main frequency of the periodic structured noise

### Main frequency of the periodic structured noise 
  
The main frequency needs to be detected for the PerStruc-Denoiser to work effectively. Therefore, the line-wise spectrum of an example image (see below) of the recording needs to be inspected. The main components of periodic structured noise should be in the middle between left side and center of the spectrum and is clearly visible as a peak.

### Frequencies of remaining high frequency structured noise

There are some recordings, which contain high frequencies structured noise. This is visible as a peak close to the center of the line spectrum (see below) and can be effectively removed with an additional gaussian ($\sigma_{mask} = 1\ pixel$).

In [ ]:
img_ft_line = utils.line_fft(file_example_denoised)

plt.figure(figsize=(20,15))
plt.imshow(np.log(np.abs(img_ft_line)), cmap='magma')
plt.rc('xtick', labelsize=18)
plt.show()

In [ ]:
# Select a x-range to detected the exact x-position of the main frequency of the periodic structured noise #
x_range = [60, 80]
#**********************************************************************************************************#

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(np.median(np.log(np.abs(img_ft_line)), axis=0))
plt.xlim([x_range[0], x_range[1]])
plt.rc('xtick', labelsize=18)
plt.show()

In [ ]:
# Select the exact x-position of the main frequency of the periodic structured noise #
PEAK_POSITION = 72
#************************************************************************************#

## Apply PerStruc-Denoiser

In [ ]:
file_denoised_pp = perstruc_denoiser(file_denoised, PEAK_POSITION)

In [ ]:
plt.figure(figsize=(30, 10))

plt.subplot(311)
plt.title("Example image (Raw)")
plt.imshow(file_raw[file_raw.shape[0]//2], cmap="Greens")


plt.subplot(312)
plt.title("Example image (3PM-N2V)")
plt.imshow(file_example_denoised, cmap="Greens")

plt.subplot(313)
plt.title("Example image (3PM-N2V+PerStruc-Denoiser)")
plt.imshow(file_denoised_pp[file_denoised_pp.shape[0]//2], cmap="Greens")

plt.tight_layout
plt.show()

In [ ]:
## Line power spectrum
# Raw
freqs, line_spectrum_raw = utils.line_power_spectrum(file_raw[file_raw.shape[0]//2])
# 3PM-N2V
_, line_spectrum_denoised = utils.line_power_spectrum(file_example_denoised)
# 3PM-N2V+PerStruc-Denoiser
_, line_spectrum_denoised_pp = utils.line_power_spectrum(file_denoised_pp[file_denoised_pp.shape[0]//2])

plt.figure(figsize=(30, 10))

plt.subplot(311)
plt.title("Averaged line power spectrum (Raw)")
plt.plot(freqs, line_spectrum_raw, linewidth = 5, label="Raw")
plt.ylabel("Normlized spectral power")
plt.xlabel(r"Spatial frequency [$Pixel^{-1}$]")

plt.subplot(312)
plt.title("Averaged line power spectrum (3PM-Noise2Void)")
plt.plot(freqs, line_spectrum_denoised, linewidth = 5, label="3PM-N2V")
plt.ylabel("Normlized spectral power")
plt.xlabel(r"Spatial frequency [$Pixel^{-1}$]")

plt.subplot(313)
plt.title("Averaged line power spectrum (3PM-Noise2Void+PerStruc-Denoiser)")
plt.plot(freqs, line_spectrum_denoised_pp, linewidth = 5, label="3PM-N2V+PerStruc-Denoiser")
plt.ylabel("Normlized spectral power")
plt.xlabel(r"Spatial frequency [$Pixel^{-1}$]")

plt.tight_layout
plt.show()

## Apply PerStruc-Denoiser + 1D-Gaussian-Filter

In [ ]:
file_denoised_pp_1D = perstruc_denoiser(file_denoised, PEAK_POSITION, use_gaussian=True)

In [ ]:
plt.figure(figsize=(40, 10))

plt.subplot(411)
plt.title("Example image (Raw)")
plt.imshow(file_raw[file_raw.shape[0]//2], cmap="Greens")


plt.subplot(412)
plt.title("Example image (3PM-N2V)")
plt.imshow(file_example_denoised, cmap="Greens")

plt.subplot(413)
plt.title("Example image (3PM-N2V+PerStruc-Denoiser)")
plt.imshow(file_denoised_pp[file_denoised_pp.shape[0]//2], cmap="Greens")

plt.subplot(414)
plt.title("Example image (3PM-N2V+PerStruc-Denoiser+Gaussian Filter)")
plt.imshow(file_denoised_pp_1D[file_denoised_pp_1D.shape[0]//2], cmap="Greens")

plt.tight_layout
plt.show()

In [ ]:
## Line power spectrum
# 3PM-N2V+PerStruc-Denoiser+Gaussian Filter
_, line_spectrum_denoised_pp_1D = utils.line_power_spectrum(file_denoised_pp_1D[file_denoised_pp_1D.shape[0]//2])

plt.figure(figsize=(40, 10))

plt.subplot(411)
plt.title("Averaged line power spectrum (Raw)")
plt.plot(freqs, line_spectrum_raw, linewidth = 5, label="Raw")
plt.ylabel("Normlized spectral power")
plt.xlabel(r"Spatial frequency [$Pixel^{-1}$]")

plt.subplot(412)
plt.title("Averaged line power spectrum (3PM-Noise2Void)")
plt.plot(freqs, line_spectrum_denoised, linewidth = 5, label="3PM-N2V")
plt.ylabel("Normlized spectral power")
plt.xlabel(r"Spatial frequency [$Pixel^{-1}$]")

plt.subplot(413)
plt.title("Averaged line power spectrum (3PM-Noise2Void+PerStruc-Denoiser)")
plt.plot(freqs, line_spectrum_denoised_pp, linewidth = 5, label="3PM-N2V+PerStruc-Denoiser")
plt.ylabel("Normlized spectral power")
plt.xlabel(r"Spatial frequency [$Pixel^{-1}$]")

plt.subplot(414)
plt.title("Averaged line power spectrum (3PM-Noise2Void+PerStruc-Denoiser+Gaussian Filter)")
plt.plot(freqs, line_spectrum_denoised_pp_1D, linewidth = 5, label="3PM-N2V+PerStruc-Denoiser+Gaussian Filter")
plt.ylabel("Normlized spectral power")
plt.xlabel(r"Spatial frequency [$Pixel^{-1}$]")

plt.tight_layout
plt.show()

## Store the PerStruc-Denoiser results

In [ ]:
store_name_raw = os.path.join(path_results, filenames_raw[0].split(os.sep)[-1])
tifffile.imwrite(store_name_raw, file_raw)

In [ ]:
store_name_denoised = os.path.join(path_results, filenames_denoised[0].split(os.sep)[-1])
tifffile.imwrite(store_name_denoised, file_denoised)

In [ ]:
store_name_denoised_pp = os.path.join(path_results, filenames_denoised[0].split(os.sep)[-1][:-4] + '-PP.tif')
tifffile.imwrite(store_name_denoised_pp, file_denoised_pp)

In [ ]:
store_name_denoised_pp_1D = os.path.join(path_results, filenames_denoised[0].split(os.sep)[-1][:-4] + '-PP-1D.tif')
tifffile.imwrite(store_name_denoised_pp_1D, file_denoised_pp_1D)